In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
## daatset cleaneda nd splited into trainninng and testing
data=pd.read_csv("https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv",sep="\t",names=["label","text"])
data["label"]=data["label"].map({"ham":0,"spam":1})
sw=set(stopwords.words("english"))
def clean(t):
    t=t.lower()
    t=t.translate(str.maketrans("","",string.punctuation))
    return " ".join([w for w in t.split() if w not in sw])
data["text"]=data["text"].apply(clean)
X_train,X_test,y_train,y_test=train_test_split(data["text"],data["label"],test_size=0.2,random_state=42)


In [ ]:
## All 3 models trained and tested on count vectoriser

cv=CountVectorizer()
Xtr_cv=cv.fit_transform(X_train)
Xte_cv=cv.transform(X_test)
nb_cv=MultinomialNB().fit(Xtr_cv,y_train)
lr_cv=LogisticRegression(max_iter=1000).fit(Xtr_cv,y_train)
svm_cv=LinearSVC().fit(Xtr_cv,y_train)
for m in [nb_cv,lr_cv,svm_cv]:
    p=m.predict(Xte_cv)
    print(type(m).__name__,accuracy_score(y_test,p))
    print(confusion_matrix(y_test,p))
    print(classification_report(y_test,p))


MultinomialNB 0.9856502242152466
[[962   4]
 [ 12 137]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.97      0.92      0.94       149

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

LogisticRegression 0.9847533632286996
[[966   0]
 [ 17 132]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.89      0.94       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.97      1115
weighted avg       0.99      0.98      0.98      1115

LinearSVC 0.9865470852017937
[[966   0]
 [ 15 134]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.90      0.95       149

    accuracy 

In [ ]:
## All 3 models trained and tested on tf-idf
tf=TfidfVectorizer()
Xtr_tf=tf.fit_transform(X_train)
Xte_tf=tf.transform(X_test)
nb_tf=MultinomialNB().fit(Xtr_tf,y_train)
lr_tf=LogisticRegression(max_iter=1000).fit(Xtr_tf,y_train)
svm_tf=LinearSVC().fit(Xtr_tf,y_train)
for m in [nb_tf,lr_tf,svm_tf]:
    p=m.predict(Xte_tf)
    print(type(m).__name__,accuracy_score(y_test,p))
    print(confusion_matrix(y_test,p))
    print(classification_report(y_test,p))


MultinomialNB 0.9713004484304932
[[966   0]
 [ 32 117]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       966
           1       1.00      0.79      0.88       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

LogisticRegression 0.9605381165919282
[[963   3]
 [ 41 108]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       0.97      0.72      0.83       149

    accuracy                           0.96      1115
   macro avg       0.97      0.86      0.90      1115
weighted avg       0.96      0.96      0.96      1115

LinearSVC 0.989237668161435
[[965   1]
 [ 11 138]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.99      0.93      0.96       149

    accuracy  

In [6]:
fn=np.array(tf.get_feature_names_out())
for i,c in enumerate(lr_tf.coef_):
    top=np.argsort(c)[-10:]
    print("class",i,fn[top])


class 0 ['service' 'win' 'text' 'reply' 'call' 'mobile' 'free' 'claim' 'stop'
 'txt']


In [7]:
models=["MultinomialNB","LogisticRegression","LinearSVC"]
acc_cv=[accuracy_score(y_test,nb_cv.predict(Xte_cv)),
        accuracy_score(y_test,lr_cv.predict(Xte_cv)),
        accuracy_score(y_test,svm_cv.predict(Xte_cv))]
acc_tf=[accuracy_score(y_test,nb_tf.predict(Xte_tf)),
        accuracy_score(y_test,lr_tf.predict(Xte_tf)),
        accuracy_score(y_test,svm_tf.predict(Xte_tf))]
pd.DataFrame({"Model":models,"CountVectorizer_Accuracy":acc_cv,"TFIDF_Accuracy":acc_tf})


,Model,CountVectorizer_Accuracy,TFIDF_Accuracy
0,MultinomialNB,0.985650,0.971300
1,LogisticRegression,0.984753,0.960538
2,LinearSVC,0.986547,0.989238
